In [1]:
import cv2
import numpy as np

In [2]:
image = cv2.imread('Images/circle.jpeg')
cv2.imshow('Ip Img',image)
cv2.waitKey(0)

13

In [3]:
image.shape

(789, 564, 3)

In [4]:
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

In [5]:
gray.shape

(789, 564)

In [6]:
cv2.imshow('gray',gray)
cv2.waitKey(0)

32

In [20]:
for i, j in enumerate(gray):
    for k in range(564):
        if gray[i][k] > 300:
            print(i,k)
            break
    else:
        continue
    break

## CANNY: Image, min_val, max_val (thresholds: to refine edges)
###     min_val: pixels having value lower than min_val will not be considered as an edge
###     max_val: edge
###     b/w min_val & max_val: consider continuous lines and joining with max_val

In [23]:
edged = cv2.Canny(gray, 80, 100)
cv2.imshow('Canny Edges', edged)
cv2.waitKey(0)

32

# FIND CONTOURS ALGORITHM

## COUNTOURS:
    CHAIN_APPROX_NONE
    CHAIN_APPROX_SIMPLE
    
## HIERARCHY:
    RETR_LIST
    RETR_EXTERNAL
    RETR_COMP
    RETR_TREE

In [8]:
contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #cv2.CHAIN_APPROX_SIMPLE
cv2.imshow('Canny Edges After Contouring', edged)
cv2.waitKey(0)

32

In [9]:
print("Number of Contours found = " + str(len(contours)))

Number of Contours found = 3


In [10]:
# contours

In [11]:
hierarchy

array([[[ 1, -1, -1, -1],
        [ 2,  0, -1, -1],
        [-1,  1, -1, -1]]], dtype=int32)

In [12]:
cv2.drawContours(image, contours, -1, (0,255,0), thickness = 2)

cv2.imshow('Contours', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## SORTING AND APPROXIMATING CONTOURS

### Sort Contours By Area

In [13]:
def get_contour_area(contours):
    all_areas = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        all_areas.append(area)
    return all_areas

In [14]:
get_contour_area(contours)

[86821.0, 54099.0, 54188.5]

In [15]:
sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)

In [16]:
print(get_contour_area(sorted_contours))

[86821.0, 54188.5, 54099.0]


In [17]:
# Iterate over our contours and draw one at a time
for c in sorted_contours:
    cv2.drawContours(image, [c], -1, (255,0,0), 3)
    cv2.waitKey(0)
    cv2.imshow('Contours by area', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Sort Contours Left to Right

In [18]:
def x_cord_contour(contours):
    """Returns the X cordinate for the contour centroid"""
    if cv2.contourArea(contours) > 10:
        M = cv2.moments(contours)
        print(M)
        return (int(M['m10']/M['m00']))
    else:
        pass

In [19]:
# for cnt in contours:
#     print(cnt)
#     m = x_cord_contour(cnt)
#     print(m)

In [20]:
def label_contour_center(image,c):
    M = cv2.moments(c)
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
    
    cv2.circle(image,(cx,cy), 10, (0,0,255), -1)
    return image

In [21]:
orginal_image = image.copy()
original_image2 = image.copy()

In [22]:
for (i, c) in enumerate(contours):
    orig = label_contour_center(image, c)
 
cv2.imshow("Center", image)
cv2.waitKey(0)

32

In [23]:
contours_left_to_right = sorted(contours, key = x_cord_contour, reverse = False)

{'m00': 86821.0, 'm10': 24440550.5, 'm01': 52507251.0, 'm20': 8093506473.0, 'm11': 14781089457.75, 'm02': 32086512163.5, 'm30': 2961503719206.25, 'm21': 4918781921192.15, 'm12': 9032559318601.416, 'm03': 19801185520495.7, 'mu20': 1213367926.5302496, 'mu11': 32804.61682534303, 'mu02': 331390619.43807155, 'mu30': 2310073.447123406, 'mu21': 24005549418.916073, 'mu12': 4224080.676537613, 'mu03': -4798846779.589235, 'nu20': 0.1609692584804957, 'nu11': 4.35196507972029e-06, 'nu02': 0.04396333635658256, 'nu30': 1.0400730156781567e-06, 'nu21': 0.01080810837778063, 'nu12': 1.9018236555141042e-06, 'nu03': -0.0021606027496831165}
{'m00': 54099.0, 'm10': 15237568.166666666, 'm01': 19386903.666666664, 'm20': 4762605986.0, 'm11': 5460507119.083333, 'm02': 7076182403.166666, 'm30': 1606645808777.1501, 'm21': 1699380408317.3167, 'm12': 1993064258739.7166, 'm03': 2629523390099.7, 'mu20': 470780191.9222293, 'mu11': -23873.382946018817, 'mu02': 128696612.66012524, 'mu30': 5687989.104720679, 'mu21': -7332

In [24]:
for (i,c)  in enumerate(contours_left_to_right):
    cv2.drawContours(orginal_image, [c], -1, (0,0,255), 3)  
    M = cv2.moments(c)
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
    cv2.putText(orginal_image, str(i+1), (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
    cv2.imshow('Sorting Left to Right', orginal_image)
    cv2.waitKey(0)
    (x, y, w, h) = cv2.boundingRect(c)  

In [25]:
def Y_cord_contour(contours):
    """Returns the Y cordinate for the contour centroid"""
    if cv2.contourArea(contours) > 10:
        M = cv2.moments(contours)
        print(int(M['m01']/M['m00']))
        return (int(M['m01']/M['m00']))
    else:
        pass

contours_top_to_bottom = sorted(contours, key = Y_cord_contour, reverse = False)

for (i,c)  in enumerate(contours_top_to_bottom):
    print(i)
    cv2.drawContours(original_image2, [c], -1, (0,255,0), 3)  
    M = cv2.moments(c)
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
    cv2.putText(original_image2, str(i+1), (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 3)
    cv2.imshow('Sorting Top to Bottom', original_image2)
    cv2.waitKey(0)

604
358
168
0
1
2


### Using ApproxPolyDP to approximate contours as a more defined shape

In [41]:
image = cv2.imread('images/leaf.jpeg')
orig_image = image.copy()
cv2.imshow('Original Image', orig_image)
cv2.waitKey(0) 

# Grayscale and binarize
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

# Find contours 
contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
copy = image.copy()

# Iterate through each contour 
for c in contours:
    #x,y,w,h = cv2.boundingRect(c)
    #cv2.rectangle(orig_image,(x,y),(x+w,y+h),(0,0,255),2)
    cv2.drawContours(image, [c], 0, (0, 255, 0), 2)
    cv2.imshow('Bounding Rectangle', image)

cv2.waitKey(0) 

# Iterate through each contour and compute the approx contour
for c in contours:
    # Calculate accuracy as a percent of the contour perimeter
    accuracy = 0.03 * cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, accuracy, True)
    cv2.drawContours(copy, [approx], 0, (0, 255, 0), 2)
    cv2.imshow('Approx Poly DP', copy)
    
cv2.waitKey(0)   
cv2.destroyAllWindows()

2023-06-17 18:04:27.056 python[98827:1105412] +[CATransaction synchronize] called within transaction
2023-06-17 18:04:39.416 python[98827:1105412] +[CATransaction synchronize] called within transaction
2023-06-17 18:04:39.433 python[98827:1105412] +[CATransaction synchronize] called within transaction
2023-06-17 18:04:39.484 python[98827:1105412] +[CATransaction synchronize] called within transaction
2023-06-17 18:04:39.500 python[98827:1105412] +[CATransaction synchronize] called within transaction
2023-06-17 18:04:39.533 python[98827:1105412] +[CATransaction synchronize] called within transaction
2023-06-17 18:04:39.549 python[98827:1105412] +[CATransaction synchronize] called within transaction
2023-06-17 18:04:39.583 python[98827:1105412] +[CATransaction synchronize] called within transaction
2023-06-17 18:04:39.616 python[98827:1105412] +[CATransaction synchronize] called within transaction


In [42]:
import numpy as np
import cv2

image = cv2.imread('images/leaf.jpeg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

cv2.imshow('Original Image', image)
cv2.waitKey(0) 

# Threshold the image
ret, thresh = cv2.threshold(gray, 176, 255, 0)

# Find contours 
contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
# Sort Contors by area and then remove the largest frame contour
n = len(contours) - 1
contours = sorted(contours, key=cv2.contourArea, reverse=False)[:n]

# Iterate through contours and draw the convex hull
for c in contours:
    hull = cv2.convexHull(c)
    cv2.drawContours(image, [hull], 0, (255, 0, 0), 2)
    cv2.imshow('Convex Hull', image)

cv2.waitKey(0)    
cv2.destroyAllWindows()

2023-06-17 18:04:57.740 python[98827:1105412] +[CATransaction synchronize] called within transaction
